Traceback (most recent call last):
  File "/Users/szhang943/Programs/PL-lineshape-code_dev/code/main.py", line 134, in <module>
    main()
  File "/Users/szhang943/Programs/PL-lineshape-code_dev/code/main.py", line 59, in main
    _, wk, _, sk, list_delta_r = readSk_qe(pre_gs, pre_es, file_phonon, phonon_interface=phonon_interface)
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/szhang943/Programs/PL-lineshape-code_dev/code/libcalc.py", line 87, in readSk_qe
    qk = calc_qk(vecR, list_pos_i, list_pos_f, list_delta_r)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/szhang943/Programs/PL-lineshape-code_dev/code/libcalc.py", line 34, in calc_qk
    list_delta = [
                 ^
  File "/Users/szhang943/Programs/PL-lineshape-code_dev/code/libcalc.py", line 35, in <listcomp>
    {"species": atom["speciesfull"], "delta": pos}
                ~~~~^^^^^^^^^^^^^^^
KeyError: 'speciesfull'

In [23]:
import os
import numpy as np
from numpy.linalg import norm, inv

import copy
import itertools
from collections import Counter
def read_cell_and_pos_poscar(filename):
    '''
    Read the cell and positions
    '''
    with open(filename, 'r') as f:
        lines = f.readlines()
    vecR = np.asarray([[float(x) for x in line.split()] for line in lines[2:5]]).T
    atoms = lines[5].split()
    natoms = [int(x) for x in lines[6].split()]
    list_pos = []
    i = 8
    for atom, n in zip(atoms, natoms):
        for j in range(n):
            list_pos.append({"species": filter(lambda x:x.isalpha(), atom), 
                             "pos" : np.asarray([float(x) for x in lines[i].split()]),
                            "speciesfull" : atom})
            i += 1
    return vecR, list_pos

def read_cell_and_pos_qe(prefix):
    '''
    Read the cell and positions from .in file and .out file (if exists)
    Note all numbers after species (like V1, Fe1) will be removed :  to (V,Fe)
    The speceis + number is save in speciesfull

    vecR must be read from CELL_PARAMETERS from input; if that does not exists, read output instead

    :return: vecR (lattice vector in columns),  list_atom(dictionary with species and pos)
    '''
#Read cell parameters
    with open(prefix + ".in", 'r') as f:
#        print("Read from QE input")
        lines = f.readlines()

    vecR = None
    list_pos = []
    for i, line in enumerate(lines):
        if ("nat" in line):
            ar = line.split(",")
            for st2 in ar:
                if ("nat" in st2):
                    ar2 = st2.split("=")
                    nat = int(ar2[-1])
        elif ("CELL_PARAMETER" in line):
            if ("ang" not in line):
                raise ValueError("Only angstrom unit is supported")
            vecR = np.asarray([[float(x) for x in line.split()] for line in lines[i+1:i+4]]).T
        elif ("ATOMIC_POSITIONS" in line):
            unit_coord = line.split()[-1].replace("{","").replace("}","").replace("(","").replace(")","")
            if ("crystal" not in line):
                print("Only crystal coordinate is supported in QE input")
                break
            for line2 in lines[i+1:i+nat+1]:
                ar = line2.strip().split()
                pos = np.asarray([float(x) for x in ar[1:4]])
                list_pos.append({"species": filter(lambda x:x.isalpha(), ar[0]), "pos" : pos, "speciesfull" : ar[0]})

    if (os.path.exists(prefix + ".out")):
#        print("Read from QE output")
        with open(prefix + ".out", 'r') as f:
            lines = f.readlines()

        if (vecR is None):#Not found in input ; read initial structure in output
            for i, line in enumerate(lines):
                if ("celldm(1)" in line):
                    i += 4
                    break
            celldm1 = float(line[15:26])
            vecR = np.asarray([[float(x)*celldm1*Bohr2Ang for x in line[23:56].split()] for line in lines[i:i+3]]).T

        if (vecR is None):
            raise ValueError("Cannot find cell parameters information or output")

        i_start = None
        for i, line in enumerate(lines):
            if (line.startswith("Begin final coordinates")):
                i_start = i + 1
                break
            elif (line.startswith("ATOMIC_POSITIONS")):
                i_start = i - 1

        if (i_start is None):
            print("Not relax calculation, skip .out file")
        else:
        #Check if it is vc-relax
#Read cell parameters
            if ("CELL" in lines[i_start+3]):#vc-relax
                vecR = np.asarray([[float(x) for x in line.split()] for line in lines[i_start+4:i_start+7]])
                if ("Ang" not in lines[i_start+3]):#Convert unit
                    vecR *= 0.529177249
                i = i_start + 8
            else: #relax
#Read from start
                for i, line in enumerate(lines):
                    if ("celldm" in line):
                        alat = float(line.split()[1]) * 0.529177249
                        break
                vecR = np.asarray([[float(x) for x in line.split()[3:6]] for line in lines[i+4:i+7]]) * alat
                #Recover i
                i = i_start + 1
            vecR = vecR.T
            vecRi = inv(vecR)

#Read atoms
            unit_coord = lines[i].split()[-1].replace("{","").replace("}","").replace("(","").replace(")","")

            list_pos = []
            while (True):
                i += 1
                line = lines[i]
                if (line.startswith("End")) or (line.strip() == ''):
                    break
                ar = line.split()
                pos = np.asarray([float(x) for x in ar[1:4]])
                if (unit_coord == "crystal"):
                    pass
                elif (unit_coord == "angstrom"):
                    pos = np.dot(vecRi, pos)
                else:
                    raise ValueError("Unsupported unit %s" % unit_coord)
                list_pos.append({"species": filter(lambda x:x.isalpha(), ar[0]), "pos" : pos, "speciesfull" : ar[0]})
#               print(list_pos[-1])


    return vecR, list_pos


In [24]:
qe_file="../../4_NV-_QE_w_phonopy/relax-gs/relax"
vasp_file="../1_relax-gs/CONTCAR"


In [25]:
vecR, list_pos=read_cell_and_pos_qe(qe_file)
print(vecR)
print(len(list_pos))
print(list_pos[0])
print(type(list_pos[0]['species']))

[[7.13658863 0.         0.        ]
 [0.         7.13658863 0.        ]
 [0.         0.         7.13658863]]
63
{'species': <filter object at 0x110436ec0>, 'pos': array([ 0.00013605, -0.00013605, -0.00013605]), 'speciesfull': 'C'}
<class 'filter'>


In [26]:
vecR2, list_pos2=read_cell_and_pos_poscar(vasp_file)
print(vecR2)
print(len(list_pos2))
for list in list_pos2:
    print(list)

print(type(list_pos2[0]['species']))

[[7.1365881 0.        0.       ]
 [0.        7.1365881 0.       ]
 [0.        0.        7.1365881]]
63
{'species': <filter object at 0x1104366b0>, 'pos': array([1.36466785e-04, 9.99863533e-01, 9.99863533e-01]), 'speciesfull': 'C'}
{'species': <filter object at 0x1104366e0>, 'pos': array([0.62376551, 0.62285014, 0.12722841]), 'speciesfull': 'C'}
{'species': <filter object at 0x1104343a0>, 'pos': array([0.37567579, 0.8755422 , 0.1248576 ]), 'speciesfull': 'C'}
{'species': <filter object at 0x110434340>, 'pos': array([0.12608009, 0.37552567, 0.37552571]), 'speciesfull': 'C'}
{'species': <filter object at 0x110436980>, 'pos': array([0.87277159, 0.62285014, 0.37623456]), 'speciesfull': 'C'}
{'species': <filter object at 0x1104370a0>, 'pos': array([0.62447433, 0.87391989, 0.37552571]), 'speciesfull': 'C'}
{'species': <filter object at 0x110436590>, 'pos': array([0.37714989, 0.12722839, 0.37623449]), 'speciesfull': 'C'}
{'species': <filter object at 0x110437700>, 'pos': array([0.12509582, 0.6